## Functions

### Never Unpack More Than Three Variables When Functions Return Multiple Values

python支持函数多返回值，支持unpacking。

导致我也偶尔写过这样的代码(

`a, b, c, d, e, f, g, h, i = func()`

这样的代码有两个明显的问题

- 顺序很容易出错，需要对照检查，出错还很难发现。。
- 可读性较差（如果代码要求pylint检查的话，行长度限制会使得这个语句七零八落）

因此

- 返回值不要超过3个
- 否则， return a small class or namedtuple instance来替代
> transformers中就使用了这种写法


### Prefer Raising Exceptions to Returning None

return None导致问题的最常见情景是：

主过程中用`if not result`，而不是`if result is not None`来判断。而函数有正确的返回值0, 或是空字符串。

一种常见的解决方式是返回一个tuple, `(flag, result)`，但调用者很容易就会忽视掉flag, 写起来也显得冗余

因此推荐的写法就是在函数中raise, 由调用者来handle.
- 复杂的API推荐派生Exception, 实现自己的异常处理类。

In [1]:
# 一个完整的例子，包括type annotation和docstring
def careful_divide(a: float, b: float) -> float:
    """Divides a by b.

    Raises:
    ValueError: When the inputs cannot be divided.
    """
    try:
        return a / b
    except ZeroDivisionError as e:
        raise ValueError('Invalid inputs')

### Know How Closures Interact with Variable Scope

嵌套函数定义我们经常使用。

内部定义的函数，可以直接访问外层函数的变量。

但如果内部定义函数对一个外层函数含有的变量进行赋值，实际行为是在内部函数的作用域内重新创建了一个新的同名变量，而不是修改外层函数变量的值。

为此要使用nonlocal关键字 ("complementary to the `global` statement")

In [2]:
def sort_priority(numbers, group):
    found = False
    def helper(x):
        nonlocal found # 否则return的found只会是False
        if x in group:
            found = True
            return (0, x)
        return (1, x)
    numbers.sort(key=helper)
    return found

numbers = [8, 3, 1, 2, 5, 4, 7, 6]
group = [2, 3, 5, 7]
found = sort_priority(numbers, group)
print('found: ', found)
print('numbers = ', numbers)

found:  True
numbers =  [2, 3, 5, 7, 1, 4, 6, 8]


In [3]:
# 注意，nonlocal是可以用class的成员变量轻松实现的
# 因此当nonlocal比较复杂的时候，不妨用class来替代

class Sorter:
    def __init__(self, group):
        self.group = group
        self.found = False
    def __call__(self, x):
        if x in self.group:
            self.found = True
            return (0, x)
        return (1, x)
sorter = Sorter(group)
numbers.sort(key=sorter)
print('found: ', sorter.found)
print('numbers = ', numbers)

found:  True
numbers =  [2, 3, 5, 7, 1, 4, 6, 8]


### Provide Optional Behavior with Keyword Arguments

除了普遍支持的position argument外，python还支持keyword argument

keyword argument的好处主要有以下三点
- 增强了函数调用的可读性
- 若函数有多个optional argument，调用时可以通过keyword argument直接设置其中的一个或几个。
- 扩展函数参数时不需要改之前的调用。(新增的参数提供默认值,即optional参数，新的调用用keyword argument设置新增参数)

In [6]:
# keyword argument的形式有多种

def func(number, divisor):
    return number % divisor

print(func(number=20, divisor=7))

kwargs = {'number': 20, 'divisor': 7}
print(func(**kwargs))

kwargs = {'divisor': 7}
print(func(number=20, **kwargs))

def print_parameters(**kwargs):
    for k, v in kwargs.items():
        print(f'{k} = {v}')

print_parameters(alpha=1.5, beta=9)


6
6
6
